In [15]:
import numpy as np
import os
import cv2
import zipfile

In [19]:
images=[]
extracted_files = "/content/extracted_files"
for file_name in os.listdir(os.path.join(extracted_files, "Pistachio")):
  img_path = os.path.join(extracted_files, "Pistachio", file_name)
  img=cv2.imread(img_path)
  img=cv2.resize(img,(500,500))
  hist=cv2.calcHist([img],[0,1,2],None,[8,8,8],[0,256,0,256,0,256])
  cv2.normalize(hist,hist)
  histogram_features=hist.flatten()
  image_gray=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  hue_features=cv2.HuMoments(cv2.moments(image_gray)).flatten()
  current_features=np.hstack([histogram_features , hue_features])
  images.append(current_features)
images=np.array(images)


In [20]:
images

array([[ 9.93987441e-01,  5.14864660e-05,  0.00000000e+00, ...,
         7.91200099e-25,  4.64527053e-16,  1.15240493e-24],
       [ 9.93400097e-01,  9.02786851e-05,  0.00000000e+00, ...,
        -2.97743123e-25,  1.22228601e-16,  5.77462805e-26],
       [ 9.87825036e-01,  2.18587360e-04,  0.00000000e+00, ...,
        -1.01246470e-25,  6.76114099e-17,  2.32079137e-25],
       ...,
       [ 9.92688060e-01,  1.73569846e-04,  0.00000000e+00, ...,
        -1.12723065e-26,  1.36610982e-17, -7.99907652e-26],
       [ 9.89905775e-01,  8.15530148e-05,  0.00000000e+00, ...,
         1.72092277e-24,  2.92237115e-16, -1.69241877e-24],
       [ 9.94166672e-01,  6.47341221e-05,  0.00000000e+00, ...,
         3.33711488e-24,  9.88161858e-17,  4.18323248e-24]])

In [21]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
images_scaled=scaler.fit_transform(images)

In [25]:
from sklearn.ensemble import IsolationForest
iso_forest=IsolationForest(contamination=0.001)
outlier_labels_iso=iso_forest.fit_predict(images_scaled)
outliers_labels_iso=np.where(outlier_labels_iso==-1)[0]
print(outliers_labels_iso)

[ 721 1165 1246]


In [33]:
from sklearn.neighbors import LocalOutlierFactor
lof=LocalOutlierFactor(contamination=0.001)
outlier_labels_lof=lof.fit_predict(images_scaled)
outliers_labels_lof=np.where(outlier_labels_lof==-1)[0]
print(outliers_labels_lof)

[ 978 1300 1428]


In [27]:
!pip install pyod

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.8/165.8 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyod: filename=pyod-2.0.2-py3-none-any.whl size=198469 sha256=b6e19ec832a203e2c47d0ef6da58ed290e7283644b221ec38630dbf772d4ee9c
  Stored in directory: /root/.cache/pip/wheels/77/c2/20/34d1f15b41b701ba69f42a32304825810d680754d509f91391
Successfully built pyod


In [28]:
from pyod.models import abod
abod_od=abod.ABOD(n_neighbors=20 , contamination=0.001)
outlire_labels_abod=abod_od.fit_predict(images_scaled)
outlires_labels_abod=np.where(outlire_labels_abod==1)[0]
print(outlires_labels_abod)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function fit_predict is deprecated
  warnings.warn(msg, category=FutureWarning)


[1165 1300 1305]
